In [1]:
!gpustat

seven                Tue Mar  9 17:15:32 2021  440.64
[0] TITAN RTX        | 35'C,   0 % |  1660 / 24220 MB | sharuev(1649M)
[1] TITAN RTX        | 85'C,  44 % |  3308 / 24220 MB | cwb(3297M)
[2] TITAN RTX        | 33'C,   0 % |  7900 / 24220 MB | mlepekhin(7887M)
[3] TITAN RTX        | 87'C,  99 % |  9156 / 24220 MB | cwb(2917M) cwb(3053M) cwb(3175M)


In [2]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

In [4]:
ddd = pd.read_csv('/home/mlepekhin/data/ru_train')
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/en_train'))
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/new_ru_attacked_50.csv'))
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/en_attacked_50.csv')).sample(frac=1, random_state=42)
print(ddd.shape)
ddd.head()

(3909, 7)


,Unnamed: 0,target,text,changed_words_num,new_model_target,old_model_target,old_text
82,82,A12,"Теги : Cubitek Тайваньская биржа Cubitek , до ...",19.0,A7,A12,"Теги : Cubitek Тайваньская компания Cubitek , ..."
92,92,A4,"﻿ II "" А что , если уж и был обыск ? Что , дум...",11.0,A11,A4,"﻿ II "" А что , если уж и был обыск ? Что , есл..."
771,902,A4,Kingdom Hearts - Two Destinies are One Chapter...,NaN,NaN,NaN,NaN
400,400,A8,"Берзиньш душевно согласился него себя "" Такую...",27.0,A17,A8,"Берзиньш душевно принял у себя "" Новую волну ..."
450,450,A17,"Ориентироваться платные фильм "" Техасская ре...",37.0,A12,A17,"Смотреть онлайн фильм "" Техасская резня бенз..."


In [5]:
ddd.to_csv('enru_attacked_train_50.csv')

Let's download all the data.

In [6]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_xlm_roberta_enru_attacked_50' 
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [7]:
transformer_model = 'xlm-roberta-base'
MAX_TOKENS = 512

In [8]:
tokenizer = PretrainedTransformerTokenizer(transformer_model)

03102021 18:07:16|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 18:07:16|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 18:07:17|INFO|transformers.tokenization_

In [9]:
train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    "enru_attacked_train_50.csv", 
    pathjoin(DATA_DIR, "manual_genres", "all.csv"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

03102021 18:07:24|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 18:07:24|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 18:07:25|INFO|transformers.tokenization_

03102021 18:07:34|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-sentencepiece.bpe.model from cache at /home/mlepekhin/.cache/torch/transformers/0c370616ddfc06067c0634160f749c2cf9d8da2c50e03a2617ce5841c8df3b1d.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed
03102021 18:07:36|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 18:07:36|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
 

Reading data
<class 'data_processing.ClassificationDatasetReader'> enru_attacked_train_50.csv


03102021 18:13:04|INFO|allennlp.data.vocabulary| Fitting token dictionary from dataset.



Building the vocabulary


In [10]:
model = build_pool_transformer_model(vocab, transformer_model)

Building the model


03102021 18:13:05|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 18:13:05|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 18:13:05|INFO|transformers.modeling_util

In [11]:
if torch.cuda.is_available():
    cuda_device = 2
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

2


In [12]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [13]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

03102021 18:13:23|INFO|allennlp.training.optimizers| Number of trainable parameters: 278052107
03102021 18:13:23|WARNING|allennlp.training.trainer| You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
03102021 18:13:23|INFO|allennlp.training.trainer| Beginning training.
03102021 18:13:23|INFO|allennlp.training.trainer| Epoch 0/9
03102021 18:13:23|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5933.28


Starting training


03102021 18:13:23|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 18470
03102021 18:13:23|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 2662
03102021 18:13:23|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 1990
03102021 18:13:23|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9928
03102021 18:13:23|INFO|allennlp.training.trainer| Training


03102021 18:15:37|INFO|allennlp.training.trainer| Validating


03102021 18:16:18|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03102021 18:16:18|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.673  |     0.721
03102021 18:16:18|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  18470.000  |       N/A
03102021 18:16:18|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  2662.000  |       N/A
03102021 18:16:18|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  1990.000  |       N/A
03102021 18:16:18|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9928.000  |       N/A
03102021 18:16:18|INFO|allennlp.training.tensorboard_writer| loss               |     1.015  |     0.851
03102021 18:16:18|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03102021 18:16:18|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5933.280  |       N/A


...

03102021 18:40:46|INFO|allennlp.training.trainer| Epoch duration: 0:03:01.386863
03102021 18:40:46|INFO|allennlp.training.trainer| Estimated training time remaining: 0:03:02
03102021 18:40:46|INFO|allennlp.training.trainer| Epoch 9/9
03102021 18:40:46|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5933.28
03102021 18:40:46|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 18526
03102021 18:40:46|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 2662
03102021 18:40:46|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 18526
03102021 18:40:46|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9928
03102021 18:40:46|INFO|allennlp.training.trainer| Training


03102021 18:43:02|INFO|allennlp.training.trainer| Validating


03102021 18:43:42|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03102021 18:43:42|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.988  |     0.752
03102021 18:43:42|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  18526.000  |       N/A
03102021 18:43:42|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  2662.000  |       N/A
03102021 18:43:42|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  18526.000  |       N/A
03102021 18:43:42|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9928.000  |       N/A
03102021 18:43:42|INFO|allennlp.training.tensorboard_writer| loss               |     0.039  |     1.328
03102021 18:43:42|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03102021 18:43:42|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5933.280  |       N/A


03102021 18:43:46|INFO|allennlp.training.trainer| Epoch duration: 0:03:00.301953
03102021 18:43:46|INFO|allennlp.training.checkpointer| loading best weights


Finished training


In [14]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

03102021 18:43:50|WARNING|root| vocabulary serialization directory /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked_50/vocab is not empty
03102021 18:43:50|INFO|filelock| Lock 140595495586784 acquired on /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked_50/vocab/.lock
03102021 18:43:50|INFO|filelock| Lock 140595495586784 released on /home/mlepekhin/models/allennlp_xlm_roberta_enru_attacked_50/vocab/.lock


total 1,1G
-rw-rw-r-- 1 mlepekhin mlepekhin 1,1G мар 10 18:28 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K мар 10 18:13 log
